# Gradio custom Bot with LangChain ConversationEntityMemory

https://www.linkedin.com/pulse/build-qa-bot-over-private-data-openai-langchain-leo-wang/

In [17]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationEntityMemory
from langchain.chains.conversation.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain

import os
os.environ["OPENAI_API_KEY"] = ""

#llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')
#llm = OpenAI(temperature=0, model='gpt-3.5-turbo')

## Define LLM

In [2]:
# Define the LLM chat model
llm = ChatOpenAI(temperature=0)

## Data Digestion

In [3]:
from langchain.document_loaders import DirectoryLoader

pdf_loader = DirectoryLoader('./Reports/', glob="**/*.pdf")
txt_loader = DirectoryLoader('./Reports/', glob="**/*.txt")
word_loader = DirectoryLoader('./Reports/', glob="**/*.docx")

loaders = [pdf_loader, txt_loader, word_loader]
documents = []
for loader in loaders:
    documents.extend(loader.load())

print(f"Total number of documents: {len(documents)}")

Total number of documents: 1


## Text splitter

Once the data is ingested, it needs to be split into smaller chunks. By default, Tiktoken is used to count tokens for OpenAI LLMs.

You can also use it to count tokens when splitting documents.

Here we are splitting the text into 1k tokens with no overlap.

In [4]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

Created a chunk of size 1239, which is longer than the specified 1000
Created a chunk of size 1078, which is longer than the specified 1000
Created a chunk of size 1036, which is longer than the specified 1000
Created a chunk of size 1311, which is longer than the specified 1000
Created a chunk of size 1019, which is longer than the specified 1000
Created a chunk of size 1371, which is longer than the specified 1000
Created a chunk of size 1019, which is longer than the specified 1000
Created a chunk of size 1028, which is longer than the specified 1000
Created a chunk of size 1895, which is longer than the specified 1000
Created a chunk of size 1302, which is longer than the specified 1000
Created a chunk of size 1381, which is longer than the specified 1000
Created a chunk of size 1152, which is longer than the specified 1000
Created a chunk of size 1038, which is longer than the specified 1000
Created a chunk of size 1011, which is longer than the specified 1000
Created a chunk of s

## Embeddings

In [5]:
persist_dir = "chroma"
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings, persist_directory=persist_dir)
vectorstore.persist()

Using embedded DuckDB with persistence: data will be stored in: chroma


## Define ConversationalRetrievalChain with EntityMemory

In [16]:
print(ENTITY_MEMORY_CONVERSATION_TEMPLATE)

input_variables=['entities', 'history', 'input'] output_parser=None partial_variables={} template='You are an assistant to a human, powered by a large language model trained by OpenAI.\n\nYou are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nYou are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own tex

In [13]:

# Create a ConversationEntityMemory object if not already created
memory = ConversationEntityMemory(llm=llm, k=10, return_messages=True, chat_history_key='history')


In [14]:
qa = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever= vectorstore.as_retriever(), 
    memory=memory)

In [15]:
print(chain.combine_documents_chain.llm_chain.prompt.template)

NameError: name 'chain' is not defined

In [ ]:
# Define the LLM chat model
llm = ChatOpenAI(temperature=0.9)

# Create a ConversationEntityMemory object if not already created
memory = ConversationEntityMemory(llm=llm, k=10)
# combine them into a chain
conversation = ConversationChain(
    llm=llm,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=memory,
    verbose=True
)

Test the Chain:

In [ ]:
# output = conversation.predict(
#     input="""
#     John helped Max study for a math exam. 
#     Thanks to him, Max got a high grade from the exam.
#     """)
# print(output)

Here is the logic:

1. Start a new variable "chat_history" with empty string
2. Always pass the user question and history to the model
3. Append the answer to the chat history
4. Repeat

It is literally three lines of code. I had a function only because of the front end.

In [ ]:
# Front end web app
import gradio as gr
demo = gr.Blocks()
with demo:
    gr.Markdown(
        """
        # 🦜🔗 Test ChatBot with EntityMemory
        """
    )
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    
    def user(user_message, history):
        # Format the list according to the expected input by ConversationalRetrievalChain
        history = [(item[0], item[1]) for item in history]
        # Get response from QA chain (history not used here, it is already buffered)
        response = conversation.run(user_message)
        # Keep the same ouput as before avoid error in Gradio, but explicit history is not used in QA chain
        history.append((user_message, response))
        return gr.update(value=""), history
    
    msg.submit(user, inputs=[msg, chatbot], outputs=[msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(debug=True)

Now with a custom bot and streaming the bot output

In [ ]:
import gradio as gr
import random
import time
# Add presets for Gradio theme
from app_modules.presets import * 
# Add custom CSS
with open("assets/custom.css", "r", encoding="utf-8") as f:
    customCSS = f.read()

with gr.Blocks(css=customCSS, theme=small_and_beautiful_theme) as demo:
    
    gr.Markdown(
        """
        # 🦜🔗 Test Gradio ChatBot with LangChain EntityMemory
        """
    )
    
    # Start chatbot with welcome from bot
    chatbot = gr.Chatbot([(None,'How can I help you?')]).style(height=650)
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def user(user_message, history):
        return gr.update(value="", interactive=False), history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0] # get if from most recent history element
        bot_message  = conversation.run(user_message)
        history[-1][1] = ""
        for character in bot_message:
            history[-1][1] += character
            #time.sleep(0.05)
            yield history

    response = msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    response.then(lambda: gr.update(interactive=True), None, [msg], queue=False)

demo.queue()
demo.launch()

In [ ]:
conversation.memory.entity_store.store

In [ ]:
conversation.memory.

In [ ]:
conversation.memory.entity_store.store